In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
tf.__version__

'1.1.0'

In [33]:
class FFCell:
    def __init__(self, input_size, output_size, hidden_desc, regression = False):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_desc = hidden_desc
        self.regression = regression
        self.weights = []
        self.activations = []
        self.X_train = tf.placeholder(tf.float32, [None, input_size], name="X_train")
        self.y_train = tf.placeholder(tf.float32, name="y_train")
        self._init_weights()
        self._build_graph()
    
    def _init_weights(self):
        self.weights.append({ 'weights': tf.Variable(tf.random_normal([self.input_size, self.hidden_desc[0]['hidden_nodes']])),
                             'biases': tf.Variable(tf.random_normal([self.hidden_desc[0]['hidden_nodes']]))})
        
        if len(self.hidden_desc[1:]) >= 1:
            cnt = 0
            for desc in self.hidden_desc[1:]:
                i_size = self.hidden_desc[cnt]['hidden_nodes']
                self.weights.append({ 'weights': tf.Variable(tf.random_normal([i_size, desc['hidden_nodes']])),
                                     'biases': tf.Variable(tf.random_normal([desc['hidden_nodes']]))})
                cnt += 1
        
        self.weights.append({ 'weights': tf.Variable(tf.random_normal([self.hidden_desc[len(self.hidden_desc) - 1]['hidden_nodes'], self.output_size])),
                             'biases': tf.Variable(tf.random_normal([self.output_size]))})
    
    def _build_graph(self):
        activ_func = self.hidden_desc[0]['activation']
        self.activations.append(
            activ_func(tf.matmul(self.X_train, self.weights[0]['weights']) + self.weights[0]['biases'])
        )
        
        if len(self.weights[1:-1]) >= 1:
            cnt = 1
            for weight in self.weights[1:-1]:
                activ_func = self.hidden_desc[cnt]['activation']
                prev_activ = self.activations[len(self.activations) - 1]
                self.activations.append(
                    activ_func(tf.matmul(prev_activ, weight['weights']) + weight['biases'])
                )
                cnt += 1
        
        weights_len = len(self.weights)
        prev_activ = self.activations[len(self.activations) - 1]
        self.logits = tf.matmul(prev_activ, self.weights[weights_len - 1]['weights']) + self.weights[weights_len - 1]['biases']
    
    def _get_batch(self, training_data, epoch, batch_size):
        start = epoch * batch_size
        end = start + batch_size
        
        return training_data['x'][start:end], training_data['y'][start:end]
    
    def predict(self):
        self.prediction = tf.softmax(self.logits)
        return prediction
    
    def train(self, training_data, batch_size, epochs):
        # TODO do train stuff
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.y_train))
        
        optimizer = tf.train.AdamOptimizer().minimize(cost)
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for epoch in range(epochs):
                epoch_cost = 0
                for _ in range(int(training_data.train.num_examples/batch_size)):
                    x, y = training_data.train.next_batch(batch_size)
                    _, c = sess.run([optimizer, cost], feed_dict = {self.X_train: x, self.y_train: y})
                    epoch_cost += c
                print('Epoch: ', (epoch + 1), '/', epochs, ' Epoch cost: ', epoch_cost)
            correct = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.y_train, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            print('Accuracy', accuracy.eval({self.X_train:training_data.test.images,
                                             self.y_train:training_data.test.labels}))
    

In [40]:
mnist = input_data.read_data_sets('../Data/MNIST/', one_hot=True)

hidden_desc = [
    {'hidden_nodes': 120, 'activation': tf.nn.relu},
]

ffcell = FFCell(784, 10, hidden_desc, False)

ffcell.train(mnist, 100, 20)


Extracting ../Data/MNIST/train-images-idx3-ubyte.gz
Extracting ../Data/MNIST/train-labels-idx1-ubyte.gz
Extracting ../Data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ../Data/MNIST/t10k-labels-idx1-ubyte.gz
('Epoch: ', 1, '/', 20, ' Epoch cost: ', 10694.519583702087)
('Epoch: ', 2, '/', 20, ' Epoch cost: ', 2505.2157199382782)
('Epoch: ', 3, '/', 20, ' Epoch cost: ', 1658.5845910608768)
('Epoch: ', 4, '/', 20, ' Epoch cost: ', 1237.5739611685276)
('Epoch: ', 5, '/', 20, ' Epoch cost: ', 968.40374245494604)
('Epoch: ', 6, '/', 20, ' Epoch cost: ', 783.63975195214152)
('Epoch: ', 7, '/', 20, ' Epoch cost: ', 646.9336726218462)
('Epoch: ', 8, '/', 20, ' Epoch cost: ', 543.53895842656493)
('Epoch: ', 9, '/', 20, ' Epoch cost: ', 460.06691773241619)
('Epoch: ', 10, '/', 20, ' Epoch cost: ', 393.83007322996855)
('Epoch: ', 11, '/', 20, ' Epoch cost: ', 337.11191564658657)
('Epoch: ', 12, '/', 20, ' Epoch cost: ', 289.54601872852072)
('Epoch: ', 13, '/', 20, ' Epoch cost: ', 249.53311148192734